In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

# Step 1: Data Loading
# Load genetic variant data (1000 Genomes Project)
# Load gene expression data (GEUVADIS)
# Assuming data is in CSV format for this example
genetic_data = pd.read_csv('1000_genomes_data.csv')
expression_data = pd.read_csv('geuvadis_expression_data.csv')

# Step 2: Data Preprocessing
# Merge datasets on a common identifier
data = pd.merge(genetic_data, expression_data, on='SampleID')

# Handling missing values
data = data.dropna()

# Standardizing the feature set
X = data.drop(columns=['SampleID', 'GeneExpression'])
y = data['GeneExpression']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 4: Feature Importance Calculation

# Pearson Correlation
pearson_corr = [pearsonr(X_train[:, i], y_train)[0] for i in range(X_train.shape[1])]
pearson_corr = np.abs(pearson_corr)

# KNN Correlation
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
knn_importances = np.abs(knn.feature_importances_ if hasattr(knn, "feature_importances_") else knn.coef_)

# Elastic Net Correlation
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(X_train, y_train)
elastic_net_importances = np.abs(elastic_net.coef_)

# Random Forest Correlation
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_importances = np.abs(rf.feature_importances_)

# Combine feature importances into a DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Pearson': pearson_corr,
    'KNN': knn_importances,
    'ElasticNet': elastic_net_importances,
    'RandomForest': rf_importances
})

# Step 5: Analyze and Visualize Feature Importance
# Normalizing importance scores
feature_importance_df[['Pearson', 'KNN', 'ElasticNet', 'RandomForest']] = feature_importance_df[['Pearson', 'KNN', 'ElasticNet', 'RandomForest']].apply(lambda x: x / x.max())

# Calculate the average importance score
feature_importance_df['Average'] = feature_importance_df[['Pearson', 'KNN', 'ElasticNet', 'RandomForest']].mean(axis=1)

# Sort by average importance
feature_importance_df = feature_importance_df.sort_values(by='Average', ascending=False)

# Display top features
print("Top features based on average importance:")
print(feature_importance_df.head(10))

# Visualize feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x='Average', y='Feature', data=feature_importance_df.head(20))
plt.title('Top 20 Features based on Average Importance')
plt.show()

# Step 6: Feature Selection based on Importance
# Select the most important SNPs
top_snps = feature_importance_df['Feature'].head(50)  # Top 50 SNPs

# Re-train models using only the top SNPs
X_train_top = X_train[:, feature_importance_df.index[:50]]
X_test_top = X_test[:, feature_importance_df.index[:50]]

# Re-train SVM
svm = SVR()
svm_params = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10]}
svm_grid = GridSearchCV(svm, svm_params, cv=5, scoring='r2')
svm_grid.fit(X_train_top, y_train)
y_pred_svm_top = svm_grid.predict(X_test_top)
print("SVM R2 Score with Top SNPs:", r2_score(y_test, y_pred_svm_top))

# Re-train Random Forest
rf_grid.fit(X_train_top, y_train)
y_pred_rf_top = rf_grid.predict(X_test_top)
print("Random Forest R2 Score with Top SNPs:", r2_score(y_test, y_pred_rf_top))

# Re-train Elastic Net
elastic_net_grid.fit(X_train_top, y_train)
y_pred_elastic_net_top = elastic_net_grid.predict(X_test_top)
print("Elastic Net R2 Score with Top SNPs:", r2_score(y_test, y_pred_elastic_net_top))

# Evaluate model performance with selected SNPs
print("Mean Squared Error for SVM with Top SNPs:", mean_squared_error(y_test, y_pred_svm_top))
print("Mean Squared Error for Random Forest with Top SNPs:", mean_squared_error(y_test, y_pred_rf_top))
print("Mean Squared Error for Elastic Net with Top SNPs:", mean_squared_error(y_test, y_pred_elastic_net_top))

